In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime 
import seaborn as sns
%matplotlib inline

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.base import TransformerMixin

In [3]:
def set_data(file):
    bikeshare_machine = pd.read_csv(file, 
                        parse_dates=['Start date', 'End date', 'start_date_short', 'end_date_short'])
    bikeshare_machine.drop('Unnamed: 0', 1, inplace=True)
    included_cols = ['start_station','end_station','Member Type','time_diff','season','mnth','holiday',
                     'weekday','workingday','weathersit','temp','hum','windspeed','miles',
                     'rush_hour','metro_dist']
    bikeshare_machine = bikeshare_machine[included_cols]
    bikeshare_machine['season'] = bikeshare_machine['season'].astype('category')
    bikeshare_machine['mnth'] = bikeshare_machine['mnth'].astype('category')
    bikeshare_machine['holiday'] = bikeshare_machine['holiday'].astype('category')
    bikeshare_machine['weekday'] = bikeshare_machine['weekday'].astype('category')
    bikeshare_machine['workingday'] = bikeshare_machine['workingday'].astype('category')
    bikeshare_machine['weathersit'] = bikeshare_machine['weathersit'].astype('category')
    bikeshare_machine['Member Type'] = bikeshare_machine['Member Type'].astype('category')
    bikeshare_machine['start_station'] = bikeshare_machine['start_station'].astype('category')
    bikeshare_machine['end_station'] = bikeshare_machine['end_station'].astype('category')
    bikeshare_machine['rush_hour'] = bikeshare_machine['rush_hour'].astype('category')
    col_names = ['start_station', 'end_station','member_type','time_diff','season','month','holiday',
             'weekday','work_day','weather_cat','temperature','humidity','windspeed','miles','rush_hour',
                'metro_dist']
    bikeshare_machine.columns = col_names
    tmin = -8
    tmax = 39
    hum_max = 100
    wind_max = 67
    bikeshare_machine['temp'] = bikeshare_machine['temperature'] * (tmax - tmin) + tmin
    bikeshare_machine['hum'] = bikeshare_machine['humidity'] * 100
    bikeshare_machine['wind'] = bikeshare_machine['windspeed'] * 67
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['rush_hour','member_type','holiday','work_day'], drop_first=True)
    bikeshare_machine = pd.get_dummies(bikeshare_machine, 
                                 columns=['start_station','end_station','season','month','weekday','weather_cat'])
    bikeshare_machine = shuffle(bikeshare_machine)
    return bikeshare_machine

In [4]:
file = '/Users/matthewcassi/Documents/Bike-Sharing-Dataset/Bikeshare_Time_Prediction/reg_metro_rush.csv'
bikeshare_machine = set_data(file)

In [5]:
bikeshare_machine = bikeshare_machine.drop(['temperature', 'humidity', 'windspeed'], 1)
bikeshare_machine = bikeshare_machine.rename(columns = {'member_type_Registered':'member_type'})
bikeshare_machine.head()

,time_diff,miles,metro_dist,temp,hum,wind,rush_hour_1,holiday_1,work_day_1,start_station_10th & Monroe St NE,...,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weather_cat_1,weather_cat_2,weather_cat_3
841619,13.967,1.232617,0.133167,14.050849,50.7917,12.667489,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
515906,16.417,1.188231,0.133167,28.503349,65.0417,8.750200,0,0,1,0,...,0,0,1,0,0,0,0,1,0,0
829843,20.683,2.639456,0.961098,6.844151,54.3333,14.125543,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0
47308,7.200,1.125802,0.252381,2.421733,42.3043,6.305571,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
370803,9.917,0.495632,0.152939,23.646651,69.7083,11.209100,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0


### Model 1

In [6]:
X1 = bikeshare_machine[['miles']]
y1 = bikeshare_machine['time_diff']

In [7]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((762165, 1), (254056, 1), (762165,), (254056,))

In [8]:
model1 = LinearRegression()
model1.fit(X1_train, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [9]:
pred1 = model1.predict(X1_test)
model1_r = model1.score(X1_test, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [10]:
model1_r, adjustedr1, model1_mse, model1_rmse

(0.41799867844373595,
 0.41799638758698288,
 37.651639452068494,
 6.1360931749826371)

### Model 2

In [11]:
X1 = bikeshare_machine[['miles', 'temp']]
y1 = bikeshare_machine['time_diff']

In [12]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((762165, 2), (254056, 2), (762165,), (254056,))

In [13]:
model1 = LinearRegression()
model1.fit(X1_train, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [14]:
pred1 = model1.predict(X1_test)
model1_r = model1.score(X1_test, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [15]:
model1_r, adjustedr1, model1_mse, model1_rmse

(0.42365599692219713,
 0.42365145972717821,
 37.194277497098248,
 6.098711134092043)

### Model 3

In [16]:
X1 = bikeshare_machine[['miles', 'holiday_1', 'work_day_1']]
y1 = bikeshare_machine['time_diff']

In [17]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((762165, 3), (254056, 3), (762165,), (254056,))

In [18]:
model1 = LinearRegression()
model1.fit(X1_train, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [19]:
pred1 = model1.predict(X1_test)
model1_r = model1.score(X1_test, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [20]:
model1_r, adjustedr1, model1_mse, model1_rmse

(0.42163277666177956,
 0.42162594695105104,
 37.509372964075624,
 6.1244896084551916)

### Model 4

In [21]:
remove_cols = ['time_diff','hum','temp','wind']

In [22]:
X1 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y1 = bikeshare_machine['time_diff']

In [27]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((762165, 293), (254056, 293), (762165,), (254056,))

In [28]:
model1 = LinearRegression()
model1.fit(X1_train, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [29]:
pred1 = model1.predict(X1_test)
model1_r = model1.score(X1_test, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test.shape[1]-1)

In [30]:
model1_r, adjustedr1, model1_mse, model1_rmse

(0.45398644966232515,
 0.45335600865756898,
 35.054761829255014,
 5.920706193458261)

### Model 5

In [31]:
class CustomScaler(TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
    
    def fit(self, X, y):
        self.scaler.fit(X[:, 0:2], y)
        return self
    
    def transform(self, X):
        X_head = self.scaler.transform(X[:, 0:2])
        return np.concatenate((X_head, X[:, 2:]), axis=1)

In [32]:
remove_cols = ['time_diff','hum','temp','wind']
X1 = np.matrix(bikeshare_machine.drop(remove_cols, 1))
y1 = bikeshare_machine['time_diff']

In [38]:
# Split the data into training and testing sets and check the shape
X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size = 0.25)
X1_train.shape, X1_test.shape, y1_train.shape, y1_test.shape

((762165, 293), (254056, 293), (762165,), (254056,))

In [39]:
scale = CustomScaler()
scale.fit(X1_train, y1_train)
X1_train_scale = scale.transform(X1_train)
X1_test_scale = scale.transform(X1_test)

In [40]:
model1 = LinearRegression()
model1.fit(X1_train_scale, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [41]:
pred1 = model1.predict(X1_test_scale)
model1_r = model1.score(X1_test_scale, y1_test)
model1_mse = mean_squared_error(y1_test, pred1)
model1_rmse = np.sqrt(model1_mse)
adjustedr1 = 1 - (1-model1_r)*(len(y1_test)-1)/(len(y1_test)-X1_test_scale.shape[1]-1)

In [42]:
model1_r, adjustedr1, model1_mse, model1_rmse

(0.45453678727095503,
 0.4539069816998702,
 35.561871136082516,
 5.9633774940114694)